# Creación de 3 modelos

### Librerías a utilizar

In [5]:
import pandas as pd
import numpy as np
import pickle
import json
from datetime import datetime

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    mean_squared_error,
    mean_absolute_error,
    r2_score
)

import warnings
warnings.filterwarnings('ignore')

### Carga de datos (preprocesados) a utilizar

In [6]:
print("\n[1/6] Carga de datos...")

df_accidentes = pd.read_csv('./data_clean/data_accidentes_anio_depto.csv')
df_dia_hora = pd.read_csv('./data_clean/data_accidentes_dia_hora.csv')
df_tipos = pd.read_csv('./data_clean/data_accidentes_tipo_mes.csv')
df_lesionados = pd.read_csv('./data_clean/data_lesionados_anio_depto.csv')
df_fallecidos = pd.read_csv('./data_clean/data_fallecidos_anio_depto.csv')

print(f"Accidentes: {len(df_accidentes)} registros")
print(f"Día/Hora: {len(df_dia_hora)} registros")
print(f"Tipos: {len(df_tipos)} registros")
print(f"Lesionados: {len(df_lesionados)} registros")
print(f"Fallecidos: {len(df_fallecidos)} registros")


[1/6] Carga de datos...
Accidentes: 110 registros
Día/Hora: 168 registros
Tipos: 108 registros
Lesionados: 110 registros
Fallecidos: 110 registros


## Modelo \#1: Clasificación de Gravedad de Accidente

In [7]:
print("\n[2/6] Preparando datos para Modelo 1...")

df_modelo1 = df_accidentes.merge(
    df_lesionados,
    on=['departamento', 'año'],
    how='left'
)
df_modelo1 = df_modelo1.merge(
    df_fallecidos,
    on=['departamento', 'año'],
    how='left'
)

df_modelo1['tasa_mortalidad'] = df_modelo1['fallecidos'] / df_modelo1['accidentes']
mediana_mortalidad = df_modelo1['tasa_mortalidad'].median()
df_modelo1['alta_gravedad'] = (df_modelo1['tasa_mortalidad'] > mediana_mortalidad).astype(int)

le_depto = LabelEncoder()
df_modelo1['departamento_encoded'] = le_depto.fit_transform(df_modelo1['departamento'])
df_modelo1['tasa_lesionados'] = df_modelo1['lesionados'] / df_modelo1['accidentes']

X1 = df_modelo1[['año', 'departamento_encoded', 'accidentes', 'lesionados', 'tasa_lesionados']]
y1 = df_modelo1['alta_gravedad']

X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1, y1, test_size=0.25, random_state=42, stratify=y1
)

scaler1 = StandardScaler()
X1_train_scaled = scaler1.fit_transform(X1_train)
X1_test_scaled = scaler1.transform(X1_test)

print(f"Dataset: {len(X1)} registros")
print(f"Train: {len(X1_train)} | Test: {len(X1_test)}")
print(f"Distribución: {y1.value_counts().to_dict()}")

print("\n   Entrenando Random Forest Classifier...")
modelo1_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    class_weight='balanced'
)
modelo1_rf.fit(X1_train_scaled, y1_train)
y1_pred_rf = modelo1_rf.predict(X1_test_scaled)
y1_proba_rf = modelo1_rf.predict_proba(X1_test_scaled)[:, 1]

print("   Entrenando Gradient Boosting Classifier...")
modelo1_gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
modelo1_gb.fit(X1_train_scaled, y1_train)
y1_pred_gb = modelo1_gb.predict(X1_test_scaled)
y1_proba_gb = modelo1_gb.predict_proba(X1_test_scaled)[:, 1]

print("   Entrenando Logistic Regression...")
modelo1_lr = LogisticRegression(
    random_state=42,
    max_iter=1000,
    class_weight='balanced'
)
modelo1_lr.fit(X1_train_scaled, y1_train)
y1_pred_lr = modelo1_lr.predict(X1_test_scaled)
y1_proba_lr = modelo1_lr.predict_proba(X1_test_scaled)[:, 1]

metricas1 = {
    'Random Forest': {
        'accuracy': accuracy_score(y1_test, y1_pred_rf),
        'precision': precision_score(y1_test, y1_pred_rf),
        'recall': recall_score(y1_test, y1_pred_rf),
        'f1_score': f1_score(y1_test, y1_pred_rf),
        'roc_auc': roc_auc_score(y1_test, y1_proba_rf),
        'confusion_matrix': confusion_matrix(y1_test, y1_pred_rf).tolist()
    },
    'Gradient Boosting': {
        'accuracy': accuracy_score(y1_test, y1_pred_gb),
        'precision': precision_score(y1_test, y1_pred_gb),
        'recall': recall_score(y1_test, y1_pred_gb),
        'f1_score': f1_score(y1_test, y1_pred_gb),
        'roc_auc': roc_auc_score(y1_test, y1_proba_gb),
        'confusion_matrix': confusion_matrix(y1_test, y1_pred_gb).tolist()
    },
    'Logistic Regression': {
        'accuracy': accuracy_score(y1_test, y1_pred_lr),
        'precision': precision_score(y1_test, y1_pred_lr),
        'recall': recall_score(y1_test, y1_pred_lr),
        'f1_score': f1_score(y1_test, y1_pred_lr),
        'roc_auc': roc_auc_score(y1_test, y1_proba_lr),
        'confusion_matrix': confusion_matrix(y1_test, y1_pred_lr).tolist()
    }
}

print("\n   RESULTADOS MODELO 1:")
for modelo, metricas in metricas1.items():
    print(f"\n   {modelo}:")
    print(f"Accuracy:  {metricas['accuracy']:.4f}")
    print(f"Precision: {metricas['precision']:.4f}")
    print(f"Recall:    {metricas['recall']:.4f}")
    print(f"F1-Score:  {metricas['f1_score']:.4f}")
    print(f"ROC-AUC:   {metricas['roc_auc']:.4f}")


[2/6] Preparando datos para Modelo 1...
Dataset: 110 registros
Train: 82 | Test: 28
Distribución: {0: 55, 1: 55}

   Entrenando Random Forest Classifier...
   Entrenando Gradient Boosting Classifier...
   Entrenando Logistic Regression...

   RESULTADOS MODELO 1:

   Random Forest:
Accuracy:  0.6786
Precision: 0.6667
Recall:    0.7143
F1-Score:  0.6897
ROC-AUC:   0.7398

   Gradient Boosting:
Accuracy:  0.5714
Precision: 0.5556
Recall:    0.7143
F1-Score:  0.6250
ROC-AUC:   0.7398

   Logistic Regression:
Accuracy:  0.6071
Precision: 0.6154
Recall:    0.5714
F1-Score:  0.5926
ROC-AUC:   0.6378


## Modelo \#2: Predicción de Cantidad de Accidentes

In [8]:
print("\n[3/6] Preparando datos para Modelo 2...")

df_modelo2 = df_accidentes.merge(
    df_lesionados,
    on=['departamento', 'año'],
    how='left'
)
df_modelo2 = df_modelo2.merge(
    df_fallecidos,
    on=['departamento', 'año'],
    how='left'
)

le_depto2 = LabelEncoder()
df_modelo2['departamento_encoded'] = le_depto2.fit_transform(df_modelo2['departamento'])

df_modelo2['año_normalizado'] = (df_modelo2['año'] - df_modelo2['año'].min()) / \
                                 (df_modelo2['año'].max() - df_modelo2['año'].min())

X2 = df_modelo2[['departamento_encoded', 'año_normalizado', 'lesionados', 'fallecidos']]
y2 = df_modelo2['accidentes']

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.25, random_state=42
)

scaler2 = StandardScaler()
X2_train_scaled = scaler2.fit_transform(X2_train)
X2_test_scaled = scaler2.transform(X2_test)

print(f"Dataset: {len(X2)} registros (usando TODOS los departamentos)")
print(f"Train: {len(X2_train)} | Test: {len(X2_test)}")

print("\n   Entrenando Random Forest Regressor...")
modelo2_rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_split=5,
    random_state=42
)
modelo2_rf.fit(X2_train_scaled, y2_train)
y2_pred_rf = modelo2_rf.predict(X2_test_scaled)

print("   Entrenando Gradient Boosting Regressor...")
from sklearn.ensemble import GradientBoostingRegressor
modelo2_gb = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
modelo2_gb.fit(X2_train_scaled, y2_train)
y2_pred_gb = modelo2_gb.predict(X2_test_scaled)

print("   Entrenando Ridge Regression...")
modelo2_ridge = Ridge(alpha=10.0, random_state=42)
modelo2_ridge.fit(X2_train_scaled, y2_train)
y2_pred_ridge = modelo2_ridge.predict(X2_test_scaled)

metricas2 = {
    'Random Forest': {
        'mse': mean_squared_error(y2_test, y2_pred_rf),
        'rmse': np.sqrt(mean_squared_error(y2_test, y2_pred_rf)),
        'mae': mean_absolute_error(y2_test, y2_pred_rf),
        'r2': r2_score(y2_test, y2_pred_rf)
    },
    'Gradient Boosting': {
        'mse': mean_squared_error(y2_test, y2_pred_gb),
        'rmse': np.sqrt(mean_squared_error(y2_test, y2_pred_gb)),
        'mae': mean_absolute_error(y2_test, y2_pred_gb),
        'r2': r2_score(y2_test, y2_pred_gb)
    },
    'Ridge Regression': {
        'mse': mean_squared_error(y2_test, y2_pred_ridge),
        'rmse': np.sqrt(mean_squared_error(y2_test, y2_pred_ridge)),
        'mae': mean_absolute_error(y2_test, y2_pred_ridge),
        'r2': r2_score(y2_test, y2_pred_ridge)
    }
}

print("\n   RESULTADOS Modelo 2:")
for modelo, metricas in metricas2.items():
    print(f"\n   {modelo}:")
    print(f"MSE:  {metricas['mse']:.2f}")
    print(f"RMSE: {metricas['rmse']:.2f}")
    print(f"MAE:  {metricas['mae']:.2f}")
    print(f"R²:   {metricas['r2']:.4f}")


[3/6] Preparando datos para Modelo 2...
Dataset: 110 registros (usando TODOS los departamentos)
Train: 82 | Test: 28

   Entrenando Random Forest Regressor...
   Entrenando Gradient Boosting Regressor...
   Entrenando Ridge Regression...

   RESULTADOS Modelo 2:

   Random Forest:
MSE:  6542.28
RMSE: 80.88
MAE:  42.45
R²:   0.9886

   Gradient Boosting:
MSE:  19996.15
RMSE: 141.41
MAE:  51.68
R²:   0.9650

   Ridge Regression:
MSE:  15037.86
RMSE: 122.63
MAE:  71.00
R²:   0.9737


## Modelo \#3: Predicción de Riesgo por Día/Hora

In [9]:
print("\n[4/6] Preparando datos para Modelo 3...")

df_modelo3 = df_dia_hora.copy()

dias_map = {
    'lunes': 1, 'martes': 2, 'miercoles': 3, 'jueves': 4,
    'viernes': 5, 'sabado': 6, 'domingo': 7
}
df_modelo3['dia_num'] = df_modelo3['dia_semana'].map(dias_map)

q25 = df_modelo3['accidentes'].quantile(0.25)
q75 = df_modelo3['accidentes'].quantile(0.75)

def clasificar_riesgo(accidentes):
    if accidentes >= q75:
        return 2
    elif accidentes >= q25:
        return 1
    else:
        return 0

df_modelo3['nivel_riesgo'] = df_modelo3['accidentes'].apply(clasificar_riesgo)

X3 = df_modelo3[['dia_num', 'hora_num']]
y3 = df_modelo3['nivel_riesgo']

X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, y3, test_size=0.30, random_state=42, stratify=y3
)

scaler3 = StandardScaler()
X3_train_scaled = scaler3.fit_transform(X3_train)
X3_test_scaled = scaler3.transform(X3_test)

print(f"Dataset: {len(X3)} registros (168 combinaciones día/hora)")
print(f"Clases: 0=Riesgo Bajo, 1=Riesgo Medio, 2=Riesgo Alto")
print(f"Train: {len(X3_train)} | Test: {len(X3_test)}")
print(f"Distribución: {pd.Series(y3).value_counts().to_dict()}")

print("\n   Entrenando Random Forest Classifier (3 niveles de riesgo)...")
modelo3_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=8,
    min_samples_split=5,
    min_samples_leaf=3,
    random_state=42,
    class_weight='balanced'
)
modelo3_rf.fit(X3_train_scaled, y3_train)
y3_pred_rf = modelo3_rf.predict(X3_test_scaled)

# Entrenar Gradient Boosting
print("   Entrenando Gradient Boosting Classifier (3 niveles de riesgo)...")
modelo3_gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=4,
    random_state=42
)
modelo3_gb.fit(X3_train_scaled, y3_train)
y3_pred_gb = modelo3_gb.predict(X3_test_scaled)

# Entrenar Decision Tree (más interpretable)
print("   Entrenando Decision Tree Classifier (3 niveles de riesgo)...")
from sklearn.tree import DecisionTreeClassifier
modelo3_dt = DecisionTreeClassifier(
    max_depth=6,
    min_samples_split=5,
    min_samples_leaf=3,
    random_state=42,
    class_weight='balanced'
)
modelo3_dt.fit(X3_train_scaled, y3_train)
y3_pred_dt = modelo3_dt.predict(X3_test_scaled)

# Métricas Modelo 3
metricas3 = {
    'Random Forest': {
        'accuracy': accuracy_score(y3_test, y3_pred_rf),
        'precision': precision_score(y3_test, y3_pred_rf, average='weighted', zero_division=0),
        'recall': recall_score(y3_test, y3_pred_rf, average='weighted'),
        'f1_score': f1_score(y3_test, y3_pred_rf, average='weighted'),
        'confusion_matrix': confusion_matrix(y3_test, y3_pred_rf).tolist()
    },
    'Gradient Boosting': {
        'accuracy': accuracy_score(y3_test, y3_pred_gb),
        'precision': precision_score(y3_test, y3_pred_gb, average='weighted', zero_division=0),
        'recall': recall_score(y3_test, y3_pred_gb, average='weighted'),
        'f1_score': f1_score(y3_test, y3_pred_gb, average='weighted'),
        'confusion_matrix': confusion_matrix(y3_test, y3_pred_gb).tolist()
    },
    'Decision Tree': {
        'accuracy': accuracy_score(y3_test, y3_pred_dt),
        'precision': precision_score(y3_test, y3_pred_dt, average='weighted', zero_division=0),
        'recall': recall_score(y3_test, y3_pred_dt, average='weighted'),
        'f1_score': f1_score(y3_test, y3_pred_dt, average='weighted'),
        'confusion_matrix': confusion_matrix(y3_test, y3_pred_dt).tolist()
    }
}

print("\n   RESULTADOS Modelo 3:")
for modelo, metricas in metricas3.items():
    print(f"\n   {modelo}:")
    print(f"      Accuracy:  {metricas['accuracy']:.4f}")
    print(f"      Precision: {metricas['precision']:.4f}")
    print(f"      Recall:    {metricas['recall']:.4f}")
    print(f"      F1-Score:  {metricas['f1_score']:.4f}")

from collections import Counter
print("\n   Distribución de clases:")
print(f"      Real:      {Counter(y3_test)}")
print(f"      Predicho:  {Counter(y3_pred_rf)}")


[4/6] Preparando datos para Modelo 3...
Dataset: 168 registros (168 combinaciones día/hora)
Clases: 0=Riesgo Bajo, 1=Riesgo Medio, 2=Riesgo Alto
Train: 117 | Test: 51
Distribución: {1: 87, 2: 43, 0: 38}

   Entrenando Random Forest Classifier (3 niveles de riesgo)...
   Entrenando Gradient Boosting Classifier (3 niveles de riesgo)...
   Entrenando Decision Tree Classifier (3 niveles de riesgo)...

   RESULTADOS Modelo 3:

   Random Forest:
      Accuracy:  0.6667
      Precision: 0.6833
      Recall:    0.6667
      F1-Score:  0.6705

   Gradient Boosting:
      Accuracy:  0.6078
      Precision: 0.6164
      Recall:    0.6078
      F1-Score:  0.6055

   Decision Tree:
      Accuracy:  0.5686
      Precision: 0.5772
      Recall:    0.5686
      F1-Score:  0.5686

   Distribución de clases:
      Real:      Counter({1: 26, 2: 13, 0: 12})
      Predicho:  Counter({np.int64(1): 22, np.int64(0): 15, np.int64(2): 14})


## Guardado de modelos entrenados

In [10]:
print("\n[5/6] Guardando modelos entrenados...")

# Modelo 1: Clasificación de Gravedad
with open('./models/modelo1_gravedad.pkl', 'wb') as f:
    pickle.dump({
        'random_forest': modelo1_rf,
        'gradient_boosting': modelo1_gb,
        'logistic_regression': modelo1_lr,
        'scaler': scaler1,
        'label_encoder_depto': le_depto,
        'feature_names': list(X1.columns),
        'metricas': metricas1
    }, f)
print("   Modelo 1 guardado: ./models/modelo1_gravedad.pkl")

# Modelo 2: Regresión de Cantidad (CORREGIDO)
with open('./models/modelo2_cantidad.pkl', 'wb') as f:
    pickle.dump({
        'random_forest': modelo2_rf,
        'gradient_boosting': modelo2_gb,
        'ridge': modelo2_ridge,
        'scaler': scaler2,
        'label_encoder_depto': le_depto2,
        'feature_names': list(X2.columns),
        'metricas': metricas2
    }, f)
print("   Modelo 2 guardado: ./models/modelo2_cantidad.pkl")

# Modelo 3: Clasificación de Nivel de Riesgo (NUEVO ENFOQUE)
with open('./models/modelo3_tipo.pkl', 'wb') as f:
    pickle.dump({
        'random_forest': modelo3_rf,
        'gradient_boosting': modelo3_gb,
        'decision_tree': modelo3_dt,
        'scaler': scaler3,
        'dias_map': dias_map,
        'niveles_riesgo': ['Bajo', 'Medio', 'Alto'],
        'feature_names': list(X3.columns),
        'metricas': metricas3
    }, f)
print("   Modelo 3 guardado: ./models/modelo3_tipo.pkl")


[5/6] Guardando modelos entrenados...
   Modelo 1 guardado: ./models/modelo1_gravedad.pkl
   Modelo 2 guardado: ./models/modelo2_cantidad.pkl
   Modelo 3 guardado: ./models/modelo3_tipo.pkl


## Resumen de metricas por modelo

In [11]:
print("\n[6/6] Generando reporte de entrenamiento...")

resumen = {
    'fecha_entrenamiento': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'version': 'CORREGIDA',
    'modelo1_gravedad': {
        'descripcion': 'Clasificación binaria: Predice si un accidente tendrá alta gravedad (alta tasa de mortalidad)',
        'algoritmos': ['Random Forest', 'Gradient Boosting', 'Logistic Regression'],
        'features': list(X1.columns),
        'n_samples': len(X1),
        'metricas': metricas1
    },
    'modelo2_cantidad': {
        'descripcion': 'Regresión: Predice la cantidad de accidentes por departamento',
        'cambios': 'Usa 110 registros (departamentos x años) en vez de 4 agregados',
        'algoritmos': ['Random Forest', 'Gradient Boosting', 'Ridge Regression'],
        'features': list(X2.columns),
        'n_samples': len(X2),
        'metricas': metricas2
    },
    'modelo3_tipo': {
        'descripcion': 'Clasificación de nivel de riesgo: Predice si un momento del día/semana tiene riesgo Bajo/Medio/Alto',
        'cambios': 'Cambio completo de enfoque: usa 168 registros (día/hora) en vez de 60 (tipos). Evita overfitting.',
        'algoritmos': ['Random Forest', 'Gradient Boosting', 'Decision Tree'],
        'features': list(X3.columns),
        'n_samples': len(X3),
        'clases': ['Bajo', 'Medio', 'Alto'],
        'metricas': metricas3
    }
}

with open('./models/resumen_modelos.json', 'w', encoding='utf-8') as f:
    json.dump(resumen, f, indent=2, ensure_ascii=False)

print("Resumen guardado: ./models/resumen_modelos.json")

print("\nMEJOR MODELO POR CATEGORÍA:")

# Mejor modelo 1
mejor_m1 = max(metricas1.items(), key=lambda x: x[1]['f1_score'])
print(f"\n   Modelo 1 (Gravedad): {mejor_m1[0]}")
print(f"      F1-Score: {mejor_m1[1]['f1_score']:.4f}")

# Mejor modelo 2
mejor_m2 = min(metricas2.items(), key=lambda x: x[1]['rmse'])
print(f"\n   Modelo 2 (Cantidad): {mejor_m2[0]}")
print(f"      RMSE: {mejor_m2[1]['rmse']:.2f}")
print(f"      R²:   {mejor_m2[1]['r2']:.4f}")

# Mejor modelo 3
mejor_m3 = max(metricas3.items(), key=lambda x: x[1]['f1_score'])
print(f"\n   Modelo 3 (Tipo): {mejor_m3[0]}")
print(f"      F1-Score: {mejor_m3[1]['f1_score']:.4f}")


[6/6] Generando reporte de entrenamiento...
Resumen guardado: ./models/resumen_modelos.json

MEJOR MODELO POR CATEGORÍA:

   Modelo 1 (Gravedad): Random Forest
      F1-Score: 0.6897

   Modelo 2 (Cantidad): Random Forest
      RMSE: 80.88
      R²:   0.9886

   Modelo 3 (Tipo): Random Forest
      F1-Score: 0.6705
